In [7]:
from models.GCN import GCN
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from time import perf_counter

In [8]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [9]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [10]:
model = GCN(in_feats, 16, n_classes, 2, activation, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [11]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [12]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [13]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.0990, Train: 0.3333, Val: 0.4140, Test: 0.4070
Epoch 02, Loss: 1.0947, Train: 0.8667, Val: 0.6800, Test: 0.6710
Epoch 03, Loss: 1.0876, Train: 0.5833, Val: 0.5420, Test: 0.5720
Epoch 04, Loss: 1.0777, Train: 0.7167, Val: 0.6240, Test: 0.6480
Epoch 05, Loss: 1.0654, Train: 0.9000, Val: 0.6920, Test: 0.7120
Epoch 06, Loss: 1.0557, Train: 0.9000, Val: 0.7280, Test: 0.7150
Epoch 07, Loss: 1.0412, Train: 0.9333, Val: 0.7480, Test: 0.7240
Epoch 08, Loss: 1.0255, Train: 0.9167, Val: 0.7380, Test: 0.7160
Epoch 09, Loss: 1.0067, Train: 0.9333, Val: 0.7280, Test: 0.7170
Epoch 10, Loss: 0.9891, Train: 0.9333, Val: 0.7220, Test: 0.7250
Epoch 11, Loss: 0.9741, Train: 0.9500, Val: 0.7440, Test: 0.7270
Epoch 12, Loss: 0.9501, Train: 0.9500, Val: 0.7500, Test: 0.7410
Epoch 13, Loss: 0.9227, Train: 0.9500, Val: 0.7500, Test: 0.7390
Epoch 14, Loss: 0.9008, Train: 0.9500, Val: 0.7540, Test: 0.7430
Epoch 15, Loss: 0.8818, Train: 0.9500, Val: 0.7600, Test: 0.7440
Epoch 16, Loss: 0.8639, T

0.786